In [ ]:
"""
An undirected graph of n nodes is defined by edgeList, where edgeList[i] = [ui, vi, disi] denotes an edge between nodes ui and vi with distance disi. Note that there may be multiple edges between two nodes.

Given an array queries, where queries[j] = [pj, qj, limitj], your task is to determine for each queries[j] whether there is a path between pj and qj such that each edge on the path has a distance strictly less than limitj .

Return a boolean array answer, where answer.length == queries.length and the jth value of answer is true if there is a path for queries[j] is true, and false otherwise.

 
Example 1:
    Input: n = 3, edgeList = [[0,1,2],[1,2,4],[2,0,8],[1,0,16]], queries = [[0,1,2],[0,2,5]]
    Output: [false,true]
    Explanation: The above figure shows the given graph. Note that there are two overlapping edges between 0 and 1 with distances 2 and 16.
    For the first query, between 0 and 1 there is no path where each distance is less than 2, thus we return false for this query.
    For the second query, there is a path (0 -> 1 -> 2) of two edges with distances less than 5, thus we return true for this query.

Example 2:
    Input: n = 5, edgeList = [[0,1,10],[1,2,5],[2,3,9],[3,4,13]], queries = [[0,4,14],[1,4,13]]
    Output: [true,false]
    Exaplanation: The above figure shows the given graph.
 

Constraints:
    2 <= n <= 105
    1 <= edgeList.length, queries.length <= 105
    edgeList[i].length == 3
    queries[j].length == 3
    0 <= ui, vi, pj, qj <= n - 1
    ui != vi
    pj != qj
    1 <= disi, limitj <= 109
    There may be multiple edges between two nodes.
    
TIP:
    1. Dynamic pre-processing.
    2. Process in order of sorted query_list and edge_list, only add 
        <edges < limit in query>;
    3. And for such queries then you just need to check if connected.
        a. Hence use of U-F DS;
"""

from typing import List
class Solution:
    def distanceLimitedPathsExist(self, n: int, edgeList: List[List[int]], queries: List[List[int]]) -> List[bool]:
        el = sorted(edgeList, key=lambda x: x[2])
        res = [False]*len(queries)
        ql = sorted([(s, e, l, i) for i, (s, e, l) in enumerate(queries)], key=lambda x: x[2])
        np = list(range(n))
        rn = [0]*n

        def findp(x):
            px = np[x]
            if px == x:
                return px
            np[x] =findp(px)
            return np[x]
        
        def union(x, y):
            px = findp(x)
            py = findp(y)
            if px == py:
                return True
            rx = rn[px]
            ry = rn[py]
            if rx < ry:
                np[px] = py
            elif ry < rx:
                np[py] = px
            else:
                np[px] = py
                rn[py] += 1

        eli = 0
        for s, e, l, i in ql:
            while eli < len(el) and el[eli][2] < l:
                u, v, w = el[eli]
                union(u, v)
                eli += 1
            res[i] = findp(s)== findp(e)
        return res

In [ ]:
from typing import List

class NodeGraph:
    def __init__(self, n):
        self.p = list(range(n))
        self.r = [0]*n
    def findp(self, x):
        px = self.p[x]
        if px == x:
            return px
        self.p[x] =self.findp(px)
        return self.p[x]
    def union(self, x, y):
        px = self.findp(x)
        py = self.findp(y)
        if px == py:
            return True
        rx = self.r[px]
        ry = self.r[py]
        if rx < ry:
            self.p[px] = py
        elif ry < rx:
            self.p[py] = px
        else:
            self.p[px] = py
            self.r[py] += 1


class Solution:
    def distanceLimitedPathsExist(self, n: int, edgeList: List[List[int]], queries: List[List[int]]) -> List[bool]:
        el = sorted(edgeList, key=lambda x: x[2])
        ql = sorted(queries, key=lambda x: x[2])

        eli = 0
        ng  = NodeGraph(n)
        res = {}
        for s, e, l in ql:
            while eli < len(el) and el[eli][2] < l:
                u, v, w = el[eli]
                ng.union(u, v)
                eli += 1
            res[(s, e, l)] = ng.findp(s)==ng.findp(e)
        result = [res[tuple(x)] for x in queries]
        return result


In [ ]:


class UVGroup:
    def __init__(self, n):
        self.p = list(range(n))
        self.r = [0]*n
        self.w = [float('-inf')]*n

    def findp(self, x):
        px = self.p[x]
        if x == px:
            return px
        self.p[x] = self.findp(px)
        return self.p[x]

    def union(self, x, y, w):
        px = self.findp(x)
        py = self.findp(y)
        if px == py:
            self.w[px] = max(self.w[px], w)
            return
        
        rx = self.r[px]
        ry = self.r[py]

        if rx < ry:
            self.p[px] = py
            self.w[py] = max(self.w[py], w)
        elif ry > rx:
            self.p[py] = px
            self.w[px] = max(self.w[px], w)
        else:
            self.p[px] = py
            self.r[px] += 1
            self.w[py] = max(self.w[py], w)
    
    def is_path(self, x, y, limit):
        px = self.findp(x)
        py = self.findp(y)
        if px != py:
            return False
        return self.w[px] < limit
    
    def __repr__(self):
        return f"{self.p}, {self.w}"

In [ ]:

# F-W APSP TLE too !!
from typing import List

class Solution:
    def distanceLimitedPathsExist(self, n: int, edgeList: List[List[int]], queries: List[List[int]]) -> List[bool]:
        maxv = float('inf')
        G = [[float('inf') for i in range(n)] for j in range(n)]
        for u, v, w in edgeList:
            G[u][v] = G[v][u] = min(w, G[u][v], G[v][u])

        for i in range(n):
            G[i][i] = 0

        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if G[i][k] == maxv or G[k][j]==maxv:
                        continue
                    ikj_max = max(G[i][k], G[k][j])
                    G[i][j] = min(G[i][j], ikj_max)
        
        result = [
            G[start][end] < limit 
            for start, end, limit in queries
        ]
        return result


In [ ]:

# TLE --- simple DFS for queries;
from typing import List

class Solution:
    def distanceLimitedPathsExist(self, n: int, edgeList: List[List[int]], queries: List[List[int]]) -> List[bool]:
        el = {}
        for u, v, w in edgeList:
            if (u, v) in el:
                el[(u, v)] = min(el[(u, v)], w)
            else:
                el[(u, v)] = w
        nal = {}
        for (x, y), w in el.items():
            if x in nal:
                nal[x].add((y, w))
            else:
                nal[x] = set([(y, w)])
            if y in nal:
                nal[y].add((x, w))
            else:
                nal[y] = set([(x, w)])
        def find_path(node, target, limit, visited):
            if node == target:
                return True
            visited.add(node)
            is_found = False
            for nbr, wt in nal.get(node, []):
                if (nbr in visited or wt >= limit):
                    continue
                is_found = find_path(nbr, target, limit, visited)
                if is_found:
                    return True
            visited.remove(node)
            return False
        result = [
            find_path(start, end, limit, set()) 
            for start, end, limit in queries
        ]
        return result


In [ ]:

# Memoization doesn't work well.
from collections import defaultdict as dd
class Solution:
    def distanceLimitedPathsExist(self, n: int, edgeList: List[List[int]], queries: List[List[int]]) -> List[bool]:
        el = {}
        for u, v, w in edgeList:
            if (u, v) in el:
                el[(u, v)] = min(el[(u, v)], w)
            else:
                el[(u, v)] = w
        nal = dd(lambda: set())
        for (x, y), w in el.items():
            nal[x].add((y, w))
            nal[y].add((x, w))

        memo = {}
        def find_path(node, target, limit, visited):
            if (node, target) in memo:
                seen_limit = memo[(node, target)]
                if limit <= seen_limit:
                    return True
            if node == target:
                memo[(node, target)] = max(memo.get((node, target), float('-inf')), limit)
                return True
            visited.add(node)
            is_found = False
            for nbr, wt in nal.get(node, []):
                if (nbr in visited or wt >= limit):
                    continue
                is_found = find_path(nbr, target, limit, visited)
                if is_found:
                    return True
            visited.remove(node)
            return False
        result = [
            find_path(start, end, limit, set()) 
            for start, end, limit in queries
        ]
        return result